In [27]:
from iroha import IrohaCrypto
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import json
from iroha_helper import *
from new_helper import *
from super_helper import *
from ipfs_functions import *
from loguru import logger

In [28]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 1

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

#for the index system
index_path = "indexdir"
index = open_dir(index_path)


In [29]:
# Customize the logger format
logger.remove()
logger.add(
    sink=lambda msg: print(msg, end=""),
    format="<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | "
           "<level>{level: <8}</level> | "
           "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - "
           "{message}",
    colorize=True,  # Enable colors for supported terminals
)

4

In [30]:
#Manually resets the index on execution
# recreate_index() 

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [31]:
hash = create_detail_contract()
integration_helpers.get_engine_receipts_result(hash)

2025-01-14 14:15:55.039 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-14 14:15:55.057 | INFO     | iroha_helper:create_detail_contract:55 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 14:15:55.061 | INFO     | iroha_helper:create_detail_contract:55 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 14:15:56.958 | INFO     | iroha_helper:create_detail_contract:55 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 14:15:56.990 | INFO     | iroha_helper:create_detail_contract:55 - ('COMMITTED', 5, 0)
2025-01-14 14:15:56.991 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-14 14:15:56.991 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-14 14:15:57.008 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-14 14:15:57.008 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


2 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` with a entry number related to an existing object in `datasets/accounts.json` and `datasets/projects.json`

4 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [32]:
# Example execution of the previous snippet
address = integration_helpers.get_engine_receipts_address(hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-01-14 14:15:57.023 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-14 14:15:57.042 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-14 14:15:57.043 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-14 14:15:57.056 | INFO     | iroha_helper:set_account_detail:296 - None
2025-01-14 14:15:57.058 | INFO     | iroha_helper:set_account_detail:298 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 14:15:57.058 | INFO     | iroha_helper:set_account_detail:298 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 14:16:00.017 | INFO     | iroha_helper:set_account_detail:298 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 14:16:00.046 | INFO     | iroha_helper:set_account_detail:298 - ('COMMITTED', 5, 0)
2025-01-14 14:16:00.047 | INFO     | iroha_helper:tracer:246 - 	Leaving "set_account_detail"
2025-01-14 14:16:00.047 | INFO     | iroha_helper:tracer:244 - 	Entering "set_acc

3 - Queries Iroha 1 for User account and checks its values

In [33]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=user_account['account_id'])
# logger.info(query)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# logger.info(response)

user_data = response.account_detail_response
user_details = user_data.detail

logger.info(f'User Account id = {user_account}, {user_details}')

2025-01-14 14:16:03.127 | INFO     | __main__:<module>:11 - User Account id = {'account_id': 'romantic_johnson@test'}, { "admin@test" : { "linked_project" : "77323@test", "user_json_ld_cid" : "QmdTLM5zBQe4CivjMxW5wXJMzErPo3qWXYvXgHHG3yTqbe" } }


6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.

In [34]:
# Process the account details response
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object
logger.info(user_details_dict)

# Now you can access the specific key like this
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(project_id)

2025-01-14 14:16:03.137 | INFO     | __main__:<module>:3 - {'admin@test': {'linked_project': '77323@test', 'user_json_ld_cid': 'QmdTLM5zBQe4CivjMxW5wXJMzErPo3qWXYvXgHHG3yTqbe'}}
2025-01-14 14:16:03.137 | INFO     | __main__:<module>:7 - 77323@test


In [35]:
schema = get_schema() #super_helper.py

logger.info(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

2025-01-14 14:16:03.148 | INFO     | __main__:<module>:3 - <Schema: ['abstract', 'created', 'creator', 'date', 'description', 'file_cid', 'format', 'full_text', 'language', 'metadata_cid', 'modified', 'project_id', 'publisher', 'subject', 'title']>
2025-01-14 14:16:03.307 | INFO     | iroha_helper:tracer:244 - 	Entering "create_contract"
2025-01-14 14:16:03.313 | INFO     | iroha_helper:create_contract:263 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-14 14:16:03.314 | INFO     | iroha_helper:create_contract:263 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-14 14:16:06.115 | INFO     | iroha_helper:create_contract:263 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-14 14:16:06.147 | INFO     | iroha_helper:create_contract:263 - ('COMMITTED', 5, 0)
2025-01-14 14:16:06.148 | INFO     | iroha_helper:tracer:246 - 	Leaving "create_contract"
2025-01-14 14:16:06.148 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-14 14:16:06.165 | INFO     | i

2025-01-13 23:09:03.420 | INFO     | iroha_helper:tracer:244 - 	Entering "create_contract"
2025-01-13 23:09:03.425 | INFO     | iroha_helper:create_contract:263 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-13 23:09:03.426 | INFO     | iroha_helper:create_contract:263 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-13 23:09:06.117 | INFO     | iroha_helper:create_contract:263 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-13 23:09:06.145 | INFO     | iroha_helper:create_contract:263 - ('COMMITTED', 5, 0)
2025-01-13 23:09:06.146 | INFO     | iroha_helper:tracer:246 - 	Leaving "create_contract"
2025-01-13 23:09:06.146 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-13 23:09:06.164 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-13 23:09:06.164 | INFO     | iroha_helper:tracer:244 - 	Entering "set_account_detail"
2025-01-13 23:09:06.178 | INFO     | iroha_helper:set_account_detail:296 - None
2025-

In [36]:
account_detail = get_account_detail(project_id)
logger.info(f"{project_id}, {account_detail}")


2025-01-14 14:16:12.978 | INFO     | __main__:<module>:2 - 77323@test, { "admin@test" : { "file_1" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, QmPCoxMyMgk689K9q4xZWpotwaeHJ4JqpQHcXthdBc9Cqe", "file_2" : "QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, QmZUche1vEJ19c314tbhPZXQWkFDDtYJeJ1nKg6pHxWwfj", "linked_user" : "romantic_johnson@test", "project_metadata_cid" : "QmaWfoLPTka6WB8qTFiJ7YhZs2k3XPkdDDocEmTS5L7ZF7" } }


In [37]:
# Perform a keyword search
keyword = "paper"
search_results, project_ids_with_cids = search_index(index, keyword)

2025-01-14 14:16:12.983 | INFO     | super_helper:search_index:230 - Starting keyword search...
2025-01-14 14:16:12.983 | INFO     | super_helper:search_index:231 - Keyword: 'paper'
2025-01-14 14:16:12.985 | INFO     | super_helper:search_index:242 - Search successful: Found 16 result(s).
2025-01-14 14:16:12.986 | INFO     | super_helper:search_index:244 - 1. Project Id: 77323@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, Metadata CID: QmZUche1vEJ19c314tbhPZXQWkFDDtYJeJ1nKg6pHxWwfj, Title: deep learning in insurance: accuracy and model interpretability using tabnet
2025-01-14 14:16:12.986 | INFO     | super_helper:search_index:244 - 2. Project Id: 77323@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q, Metadata CID: QmX8RE8fXTCPfqtPQAu7pysNUmv3BGoHf8ix4kmwVuVkS9, Title: deep learning in insurance: accuracy and model interpretability using tabnet
2025-01-14 14:16:12.986 | INFO     | super_helper:search_index:244 - 3. Project Id: 77323@test, File CID: QmV1

With Reusable Helper Function for Block Separation and Tracing

A helper function, with_logging_block, which accepts a block name, the code block to execute, and optional parameters for context-specific tracing.

Cleanly separates each logical block, improving readability, modularity, and error tracing.

Includes the output of the search results explicitly logged within the Keyword Search block. This will ensure all search results are printed for visibility and debugging purposes.

In [38]:
with with_logging_block("Keyword Search", logger):
    if not search_results:
        logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
        sys.exit(1)
    
    # Log the total number of search results
    total_results = len(search_results)
    logger.info(f"Total search results found for keyword '{keyword}': {total_results}")
    
    # List the `project_id` and `file_cid` for each search result
    logger.info("Listing all search results:")
    for idx, result in enumerate(search_results, 1):
        project_id = result.get('project_id', 'N/A')
        file_cid = result.get('file_cid', 'N/A')
        logger.info(f"Result {idx}: Project ID: {project_id}, File CID: {file_cid}")
    
    # Log the full details of search results, excluding the 'full_text' field
    logger.info(f"Full search results for keyword '{keyword}' (excluding 'full_text'):")
    for idx, result in enumerate(search_results, 1):
        filtered_result = {k: v for k, v in result.items() if k != "full_text"}
        logger.info(f"Result {idx}: {json.dumps(filtered_result, indent=2)}")


2025-01-14 14:16:13.005 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 14:16:13.005 | INFO     | super_helper:with_logging_block:34 - STARTING BLOCK: Keyword Search
2025-01-14 14:16:13.005 | INFO     | super_helper:with_logging_block:35 - ==================================================
2025-01-14 14:16:13.005 | INFO     | __main__:<module>:8 - Total search results found for keyword 'paper': 16
2025-01-14 14:16:13.005 | INFO     | __main__:<module>:11 - Listing all search results:
2025-01-14 14:16:13.005 | INFO     | __main__:<module>:15 - Result 1: Project ID: 77323@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
2025-01-14 14:16:13.005 | INFO     | __main__:<module>:15 - Result 2: Project ID: 77323@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
2025-01-14 14:16:13.005 | INFO     | __main__:<module>:15 - Result 3: Project ID: 77323@test, File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
2025-01-14 14:16:13.005 | INFO     | __main_

In [39]:
with with_logging_block("Processing Search Results", logger):
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        with with_logging_block(f"Processing Result for Project ID: {project_id or 'Unknown'}", logger):
            if not project_id or not file_cid or not metadata_cid:
                logger.error(f"Missing required data in result: {result_dict}")
                continue

            logger.info(f"File CID: {file_cid}")
            logger.info(f"Metadata CID: {metadata_cid}")

            # Fetch project details
            with with_logging_block("Fetching Project Details", logger):
                project_details = get_account_detail(project_id)
                if not project_details:
                    logger.error(f"No project details found for Project ID: {project_id}.")
                    continue
                logger.info(f"Fetched project details for {project_id}: {project_details}")

            # Parse blockchain data
            with with_logging_block("Parsing Blockchain Data", logger):
                try:
                    blockchain_data = json.loads(project_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding project details JSON for {project_id}: {e}")
                    continue

            # Validate file CID
            with with_logging_block("Validating File CID", logger):
                validation_result = fetch_project_details(file_cid, blockchain_data)
                logger.info(f"Validation Result for {project_id}: {validation_result}")
                if not validation_result["is_valid"]:
                    logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")
                    continue

            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")

            # Process project metadata
            if project_metadata_cid:
                with with_logging_block("Processing Project Metadata", logger):
                    project_metadata = download_json_from_ipfs(project_metadata_cid)
                    logger.info(f"Downloaded project metadata: {project_metadata}")

            # Process linked user details
            if linked_user:
                with with_logging_block(f"Processing Linked User: {linked_user}", logger):
                    user_details = get_account_detail(linked_user)
                    try:
                        user_details = json.loads(user_details)
                    except json.JSONDecodeError as e:
                        logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                        continue

                    user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid")
                    if user_json_ld_cid:
                        user_metadata = download_json_from_ipfs(user_json_ld_cid)
                        logger.info(f"Downloaded user metadata: {user_metadata}")
                    else:
                        logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")

            # Process metadata CID
            if metadata_cid:
                with with_logging_block("Processing Metadata CID", logger):
                    file_metadata = download_json_from_ipfs(metadata_cid)
                    file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                    logger.info(f"Downloaded file metadata: {file_metadata}")

2025-01-14 14:16:13.013 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 14:16:13.013 | INFO     | super_helper:with_logging_block:34 - STARTING BLOCK: Processing Search Results
2025-01-14 14:16:13.013 | INFO     | super_helper:with_logging_block:35 - ==================================================
2025-01-14 14:16:13.013 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 14:16:13.014 | INFO     | super_helper:with_logging_block:34 - STARTING BLOCK: Processing Result for Project ID: 77323@test
2025-01-14 14:16:13.014 | INFO     | super_helper:with_logging_block:35 - ==================================================
2025-01-14 14:16:13.014 | INFO     | __main__:<module>:12 - File CID: QmV1vYkuiwKGPA5Tyi4ECNFfWYocyTtQ3E5j7CSvPBX49Q
2025-01-14 14:16:13.014 | INFO     | __main__:<module>:13 - Metadata CID: QmZUche1vEJ19c314tbhPZXQWkFDDtYJeJ1nKg6pHxWwfj
2025-01-14 14:16:13.014 | INFO     | super_helper:with_logging_block:33 - 
2025-01-14 14:16:13.014 | INFO     |